In [107]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [108]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
# df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
# df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')

In [109]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [110]:
df_bbidding

,timestamp_utc,Mean_SolarRadiation_dwd,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_RW_dwd_Mean_1h,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,...,imbalance_price,settlement_date,settlement_period,data_provider,price_y,volume,day_ahead_price,market_price,imbalance_price_predictions,day_ahead_price_predictions
0,2021-03-04 08:00:00+00:00,50.709375,50.709375,42.008984,33.308594,15.907813,78.506250,0.194284,0.000355,5.161761,...,89.95,2021-03-04,17,APXMIDP,68.55,1201.25,73.77,68.55,51.435993,47.034920
1,2021-03-04 08:30:00+00:00,68.791812,68.791812,59.750593,50.709375,33.308594,113.314159,0.193671,0.000382,6.013144,...,89.00,2021-03-04,18,APXMIDP,69.25,1105.35,73.77,69.25,50.469364,44.789060
2,2021-03-04 09:00:00+00:00,86.874249,86.874249,77.833030,68.791812,50.709375,148.122068,0.193058,0.000454,6.864527,...,92.00,2021-03-04,19,APXMIDP,75.18,1245.05,70.86,75.18,58.320385,48.222595
3,2021-03-04 09:30:00+00:00,106.168853,106.168853,96.521551,86.874249,68.791812,186.030011,0.192385,0.000414,7.798984,...,92.00,2021-03-04,20,APXMIDP,73.19,1590.00,70.86,73.19,58.867405,48.325844
4,2021-03-04 10:00:00+00:00,125.463457,125.463457,115.816155,106.168853,86.874249,223.937954,0.191712,0.000599,8.733441,...,98.00,2021-03-04,21,APXMIDP,74.12,1415.30,63.72,74.12,61.618490,46.634870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41944,2023-08-25 23:30:00+00:00,0.009818,0.009818,0.004048,-0.001721,-0.003729,0.004615,0.187994,0.000419,13.896639,...,66.01,2023-08-26,2,APXMIDP,83.91,1201.85,84.41,83.91,94.172900,88.453670
41945,2023-08-26 00:00:00+00:00,0.021356,0.021356,0.015587,0.009818,-0.001721,0.017088,0.188246,0.000398,13.547243,...,66.01,2023-08-26,3,APXMIDP,82.13,1066.95,79.96,82.13,89.776830,80.649120
41946,2023-08-26 00:30:00+00:00,0.012875,0.012875,0.017116,0.021356,0.009818,-0.000814,0.188469,0.000386,13.237374,...,66.01,2023-08-26,4,APXMIDP,78.76,997.60,79.96,78.76,93.574730,83.318240
41947,2023-08-26 01:00:00+00:00,0.004395,0.004395,0.008635,0.012875,0.021356,-0.018716,0.188692,0.000381,12.927505,...,66.01,2023-08-26,5,APXMIDP,77.47,913.25,73.84,77.47,91.836670,78.001990


In [111]:
df_bbidding.Target_MW

0        102.844285
1        162.689737
2        172.662460
3        174.278554
4        206.460381
            ...    
41944     23.648000
41945     10.865000
41946     10.505000
41947      4.486000
41948      6.580000
Name: Target_MW, Length: 41949, dtype: float64

In [112]:
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))

In [113]:
df_bbidding.Revenue_50.mean()

25645.800237410862

50% quantil gibt historisch 25437.310022730526 pro periode

In [114]:
df_bbidding["Revenue_40"] = df_bbidding["day_ahead_price"] * df_bbidding["4"]+(df_bbidding["Target_MW"]-df_bbidding["4"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["4"]))
df_bbidding["Revenue_30"] = df_bbidding["day_ahead_price"] * df_bbidding["3"]+(df_bbidding["Target_MW"]-df_bbidding["3"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["3"]))
df_bbidding["Revenue_20"] = df_bbidding["day_ahead_price"] * df_bbidding["2"]+(df_bbidding["Target_MW"]-df_bbidding["2"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["2"]))
df_bbidding["Revenue_10"] = df_bbidding["day_ahead_price"] * df_bbidding["1"]+(df_bbidding["Target_MW"]-df_bbidding["1"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["1"]))
df_bbidding["Revenue_60"] = df_bbidding["day_ahead_price"] * df_bbidding["6"]+(df_bbidding["Target_MW"]-df_bbidding["6"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["6"]))
df_bbidding["Revenue_50"] = df_bbidding["day_ahead_price"] * df_bbidding["5"]+(df_bbidding["Target_MW"]-df_bbidding["5"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["5"]))
df_bbidding["Revenue_70"] = df_bbidding["day_ahead_price"] * df_bbidding["7"]+(df_bbidding["Target_MW"]-df_bbidding["7"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["7"]))
df_bbidding["Revenue_80"] = df_bbidding["day_ahead_price"] * df_bbidding["8"]+(df_bbidding["Target_MW"]-df_bbidding["8"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["8"]))
df_bbidding["Revenue_90"] = df_bbidding["day_ahead_price"] * df_bbidding["9"]+(df_bbidding["Target_MW"]-df_bbidding["9"])*(df_bbidding["imbalance_price"]-0.07*(df_bbidding["Target_MW"]-df_bbidding["9"]))

In [115]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_bbidding["Revenue_"+str(i)+"0"].mean())

Revenue_10
35247.63262120537
Revenue_20
32999.210119581716
Revenue_30
30541.56152004898
Revenue_40
27389.26853759782
Revenue_50
25645.800237410862
Revenue_60
22831.75377931324
Revenue_70
19894.275308889133
Revenue_80
15648.861394222415
Revenue_90
7704.361000305964


In [116]:
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [117]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Target_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



Mit optimierung 56488.5046925057 pro periode

In [118]:
df_bbidding.revenue_optimal.mean()

56847.65672443437

In [119]:
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

Revenue mit estimateten values (LSTM) Mit 50% quantile

In [120]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [121]:
df_bbidding.revenue_normal.mean()

24594.75576239479

LSTM 40% quantile

In [122]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['4']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))


In [123]:
df_bbidding.revenue_normal.mean()

26157.767641154373

On newer data

In [124]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [164]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [202]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)


In [203]:
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new.timestamp = pd.to_datetime(df_api_new.timestamp)

In [204]:

# Create a continuous time series from the minimum to maximum timestamp at 30-minute intervals
full_timestamp_range = pd.date_range(start=df_api_new['timestamp'].min(), end=df_api_new['timestamp'].max(), freq='30min')
# Reindex the dataframe using the full range of timestamps
df_api_new_1 = df_api_new.set_index('timestamp').reindex(full_timestamp_range, method=None)
df_api_new_1 = df_api_new_1.reset_index().rename(columns={'index': 'timestamp'})
# Create the 'prediction_date' column based on the timestamp
df_api_new_1['prediction_date'] = df_api_new_1['timestamp'].dt.date

In [206]:
df_api_new_1

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03 22:00:00+00:00,2024-10-03,1355.0,487.0,963.0,1544.0,1330.0,867.0,326.0,1402.0,1357.0
1,2024-10-03 22:30:00+00:00,2024-10-03,788.0,584.0,1148.0,1320.0,1007.0,565.0,923.0,797.0,487.0
2,2024-10-03 23:00:00+00:00,2024-10-03,571.0,748.0,1001.0,1460.0,740.0,1017.0,1533.0,597.0,991.0
3,2024-10-03 23:30:00+00:00,2024-10-03,1476.0,1194.0,1512.0,334.0,1054.0,1572.0,744.0,1497.0,463.0
4,2024-10-04 00:00:00+00:00,2024-10-04,1352.0,1390.0,912.0,662.0,1505.0,1093.0,814.0,487.0,1340.0
...,...,...,...,...,...,...,...,...,...,...,...
811,2024-10-20 19:30:00+00:00,2024-10-20,457.0,495.0,499.0,512.0,517.0,522.0,524.0,533.0,536.0
812,2024-10-20 20:00:00+00:00,2024-10-20,454.0,459.0,493.0,508.0,508.0,517.0,520.0,527.0,532.0
813,2024-10-20 20:30:00+00:00,2024-10-20,454.0,464.0,489.0,499.0,504.0,512.0,516.0,529.0,536.0
814,2024-10-20 21:00:00+00:00,2024-10-20,351.0,469.0,485.0,500.0,501.0,515.0,522.0,530.0,536.0


In [208]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new_1,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)


In [210]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(192)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()


In [212]:
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)

In [217]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,solar_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = pd.merge(df_api_new_merged2,wind_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = df_api_new_merged2.rename(columns={
    "generation_mw_x": "generation_solar",
    "generation_mw_y": "generation_wind"
})
df_api_new_merged2 = df_api_new_merged2.groupby("timestamp_utc").last().reset_index()
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,61.00,78.47,1627.05,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,50.00,78.47,1469.90,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,93.50,71.36,1405.55,0.0,2956.745251,2779.542388,2024-10-08,1,0.0,124.472
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,93.50,71.36,1268.95,0.0,2956.745251,2779.477654,2024-10-08,2,0.0,145.582
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,93.00,72.80,1428.80,0.0,2956.745251,2779.477632,2024-10-08,3,0.0,160.762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19.0,17.0,14.0,...,99.95,69.09,1415.55,0.0,2956.745251,2779.347644,2024-10-10,42,0.0,361.670
91,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19.0,16.0,14.0,...,10.00,69.09,1765.45,0.0,2956.745251,2779.347643,2024-10-10,43,0.0,410.810
92,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19.0,16.0,14.0,...,1.00,66.40,1913.65,0.0,2956.745251,2779.347622,2024-10-10,44,0.0,423.910
93,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19.0,17.0,14.0,...,1.00,66.40,1913.65,0.0,2956.745251,2779.347620,2024-10-10,45,0.0,417.860


In [219]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].copy()

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)
X_test = X_test.unsqueeze(1)  # Adds a sequence length dimension

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [220]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_imbalance_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imvalance_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_15448\1576695596.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_imbalance_p

In [221]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_15448\823899303.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_pr

In [222]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,volume_lag96h,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind,imvalance_price_predictions,day_ahead_price_predictions
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,1627.05,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742,79.676735,72.483940
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,1469.90,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032,78.965164,71.437645
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,1405.55,0.0,2956.745251,2779.542388,2024-10-08,1,0.0,124.472,84.008194,65.823448
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,1268.95,0.0,2956.745251,2779.477654,2024-10-08,2,0.0,145.582,80.414536,63.266029
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,1428.80,0.0,2956.745251,2779.477632,2024-10-08,3,0.0,160.762,73.853539,70.484703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19.0,17.0,14.0,...,1415.55,0.0,2956.745251,2779.347644,2024-10-10,42,0.0,361.670,97.731041,117.432167
91,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19.0,16.0,14.0,...,1765.45,0.0,2956.745251,2779.347643,2024-10-10,43,0.0,410.810,93.743729,113.997627
92,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19.0,16.0,14.0,...,1913.65,0.0,2956.745251,2779.347622,2024-10-10,44,0.0,423.910,90.337151,109.793205
93,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19.0,17.0,14.0,...,1913.65,0.0,2956.745251,2779.347620,2024-10-10,45,0.0,417.860,87.446823,107.408096


In [223]:
df_api_new_merged2["Total_MW"] = df_api_new_merged2["generation_solar"] + df_api_new_merged2["generation_wind"]

In [224]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,generation_solar,installed_capacity_mwp,capacity_mwp,settlement_date,settlement_period_y,boa,generation_wind,imvalance_price_predictions,day_ahead_price_predictions,Total_MW
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.542388,2024-10-07,47,0.0,236.742,79.676735,72.483940,236.742
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.542388,2024-10-07,48,0.0,154.032,78.965164,71.437645,154.032
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.542388,2024-10-08,1,0.0,124.472,84.008194,65.823448,124.472
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,0.0,2956.745251,2779.477654,2024-10-08,2,0.0,145.582,80.414536,63.266029,145.582
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,0.0,2956.745251,2779.477632,2024-10-08,3,0.0,160.762,73.853539,70.484703,160.762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19.0,17.0,14.0,...,0.0,2956.745251,2779.347644,2024-10-10,42,0.0,361.670,97.731041,117.432167,361.670
91,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19.0,16.0,14.0,...,0.0,2956.745251,2779.347643,2024-10-10,43,0.0,410.810,93.743729,113.997627,410.810
92,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19.0,16.0,14.0,...,0.0,2956.745251,2779.347622,2024-10-10,44,0.0,423.910,90.337151,109.793205,423.910
93,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19.0,17.0,14.0,...,0.0,2956.745251,2779.347620,2024-10-10,45,0.0,417.860,87.446823,107.408096,417.860


In [225]:
df_api_new_merged2["Revenue_40"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["4"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["4"]))
df_api_new_merged2["Revenue_30"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["3"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["3"]))
df_api_new_merged2["Revenue_20"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["2"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["2"]))
df_api_new_merged2["Revenue_10"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["1"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["1"]))
df_api_new_merged2["Revenue_60"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["6"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["6"]))
df_api_new_merged2["Revenue_50"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["5"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["5"]))
df_api_new_merged2["Revenue_70"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["7"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["7"]))
df_api_new_merged2["Revenue_80"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["8"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["8"]))
df_api_new_merged2["Revenue_90"] = df_api_new_merged2["day_ahead_price"] * df_api_new_merged2["9"]+(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"])*(df_api_new_merged2["imbalance_price"]-0.07*(df_api_new_merged2["Total_MW"]-df_api_new_merged2["9"]))

In [226]:
for i in range(1,10):
    print("Revenue_"+str(i)+"0")
    print(df_api_new_merged2["Revenue_"+str(i)+"0"].mean())

Revenue_10
19557.55751411385
Revenue_20
19363.650736507112
Revenue_30
19176.646116271357
Revenue_40
18966.44746060056
Revenue_50
18777.32964484246
Revenue_60
18574.203913812267
Revenue_70
18370.690027811055
Revenue_80
18165.994605888725
Revenue_90
17969.41410572423


In [227]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['5']
    imbalance_price = row['imvalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']))


In [228]:
df_api_new_merged2.revenue_normal.mean()

26309.94412019738

In [184]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,Revenue_30,Revenue_20,Revenue_10,Revenue_60,Revenue_50,Revenue_70,Revenue_80,Revenue_90,optimized_trade,revenue_normal
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,20075.583981,20116.491741,20156.839501,19927.996821,19970.864581,19884.569061,19818.377421,19773.549661,0.000000,19750.925781
1,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,20075.583981,20116.491741,20156.839501,19927.996821,19970.864581,19884.569061,19818.377421,19773.549661,0.000000,19750.925781
2,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,9166.269208,9252.018168,9337.207128,8861.737848,8949.446808,8773.468888,8640.015448,8550.346488,0.000000,8505.302008
3,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,9166.269208,9252.018168,9337.207128,8861.737848,8949.446808,8773.468888,8640.015448,8550.346488,0.000000,8505.302008
4,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,7693.513685,7766.185845,7838.298005,7434.751125,7509.383285,7321.752885,7207.494645,7169.128565,0.000000,7130.622485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19.0,17.0,14.0,...,28672.734977,28754.076377,28807.603977,28506.272177,28562.319777,28421.150777,28363.703177,28305.695577,150.723988,31100.012462
102,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19.0,16.0,14.0,...,13834.966173,13984.372973,14207.433173,13383.385773,13609.805973,13155.705573,13003.218773,12850.171973,155.670487,23033.130221
103,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19.0,16.0,14.0,...,-6534.279067,-6281.604267,-5903.642067,-7295.663467,-6914.341267,-7678.245667,-7934.000467,-8190.315267,149.971856,9368.903576
104,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19.0,17.0,14.0,...,14094.697028,14358.038228,14532.899028,13564.234628,13830.095828,13386.293828,13118.332628,12938.991828,153.580367,25012.641849


MLPs mit extra dmeand und margin

In [229]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        # Layer sizes from the best trial
        layer_sizes = [256, 448, 192, 96]
        dropout_rates = [0.12338360578207397, 0.2192742565593194, 0.15708417985889997, 0.253419888887539]

        # Define the layers
        self.fc1 = nn.Linear(input_dim, layer_sizes[0])
        self.fc2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.fc3 = nn.Linear(layer_sizes[1], layer_sizes[2])
        self.fc4 = nn.Linear(layer_sizes[2], layer_sizes[3])
        self.fc5 = nn.Linear(layer_sizes[3], 1)  # Output layer

        # Dropouts
        self.dropout1 = nn.Dropout(dropout_rates[0])
        self.dropout2 = nn.Dropout(dropout_rates[1])
        self.dropout3 = nn.Dropout(dropout_rates[2])
        self.dropout4 = nn.Dropout(dropout_rates[3])

        # Activation function (Swish)
        self.swish = nn.SiLU()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # First layer with ReLU
        x = self.dropout1(x)         # First dropout
        x = self.swish(self.fc2(x))  # Second layer with Swish
        x = self.dropout2(x)         # Second dropout
        x = self.swish(self.fc3(x))  # Third layer with Swish
        x = self.dropout3(x)         # Third dropout
        x = self.swish(self.fc4(x))  # Fourth layer with Swish
        x = self.dropout4(x)         # Fourth dropout
        x = self.fc5(x)              # Output layer (no activation for raw outputs)
        return x



In [230]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].copy()

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)


c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [191]:
df_api_new_merged2.columns

Index(['timestamp_utc', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period_x', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag96h',
       'imbalance_price_lag96h', 'day_ahead_price_lag1week', 'volume_lag96h',
       'generation_solar', 'installed_capacity_mwp', 'capacity_mwp',
       'settlement_date', 'settlement_period_y', 'boa', 'generation_wind',
       'imvalance_price_predictions', 'day_ahead_price_predictions',
       'Total_MW', 'Revenue_40', 'Revenue_30', 'Revenue_20', 'Revenue_10',
       'Revenue_60', 'Revenue_50', 'Revenue_70', 'Revenue_80', 'Revenue_90',
       'optimized_trade', 'revenue_normal', 'day_ahead_price_predictions_MLP',
       'imbalance_price_predictions_MLP'],
      dtype='object')

In [231]:
model_imbalance_mlp = MLP(input_dim=15)
model_imbalance_mlp.load_state_dict(torch.load("MLP_day_ahead_price.pth"))
model_imbalance_mlp.eval()
with torch.no_grad():
    predictions = model_imbalance_mlp(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions_MLP"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_15448\1782585997.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance_mlp.load_state_dict(torch.load("MLP_day_ahea

In [232]:
model_day_ahead_price = MLP(input_dim=15)
model_day_ahead_price.load_state_dict(torch.load("MLP_imbalance_price.pth"))
model_day_ahead_price.eval()
with torch.no_grad():
    predictions = model_day_ahead_price(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imbalance_price_predictions_MLP"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_15448\360064670.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_day_ahead_price.load_state_dict(torch.load("MLP_imbalan

In [233]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions_MLP']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions_MLP']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade_MLP'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade_MLP'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_MLP']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_MLP']))


In [234]:
df_api_new_merged2

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,Revenue_60,Revenue_50,Revenue_70,Revenue_80,Revenue_90,optimized_trade,revenue_normal,day_ahead_price_predictions_MLP,imbalance_price_predictions_MLP,optimized_trade_MLP
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,19927.996821,19970.864581,19884.569061,19818.377421,19773.549661,0.000000,20638.287943,100.683258,95.715752,45.482121
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,8861.737848,8949.446808,8773.468888,8640.015448,8550.346488,0.000000,10541.225648,100.786362,95.350548,48.827111
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,7434.751125,7509.383285,7321.752885,7207.494645,7169.128565,0.000000,8874.534860,96.865608,91.309464,49.686954
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,8452.595529,8533.138489,8330.731089,8207.606649,8166.285169,0.000000,10051.235098,95.580223,89.879395,50.720320
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,8906.957315,9019.087355,8831.503955,8755.490595,8678.917235,0.000000,9948.651088,97.520111,93.836006,36.314883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19.0,17.0,14.0,...,28506.272177,28562.319777,28421.150777,28363.703177,28305.695577,150.723988,30375.436829,173.635925,162.178665,91.837069
91,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19.0,16.0,14.0,...,13383.385773,13609.805973,13155.705573,13003.218773,12850.171973,155.670487,19209.862052,173.933319,162.674286,91.421403
92,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19.0,16.0,14.0,...,-7295.663467,-6914.341267,-7678.245667,-7934.000467,-8190.315267,149.971856,2800.200489,171.321136,160.108261,91.092918
93,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19.0,17.0,14.0,...,13564.234628,13830.095828,13386.293828,13118.332628,12938.991828,153.580089,20361.596274,166.552628,155.521622,89.792527


In [235]:
df_api_new_merged2.revenue_normal.mean()

21589.310000084915

Better MLP

In [236]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleModel, self).__init__()
        
        # Zwei Hidden Layers
        self.fc1 = nn.Linear(input_dim, 128)  # Erster Hidden Layer
        self.fc2 = nn.Linear(128, 64)         # Zweiter Hidden Layer
        
        # Ausgangsschicht
        self.fc3 = nn.Linear(64, 1)           # Ausgangsschicht
        
        # Dropout und Aktivierungsfunktion
        self.dropout = nn.Dropout(0.2)
        self.swish = nn.SiLU()

    def forward(self, x):
        # Durch die Hidden Layers
        x = self.swish(self.fc1(x))  # Erster Hidden Layer
        x = self.dropout(x)
        x = self.swish(self.fc2(x))  # Zweiter Hidden Layer
        x = self.dropout(x)
        
        # Ausgang
        x = self.fc3(x)  # Ausgangsschicht
        
        return x


In [237]:
df_day_ahead_demand = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/day_ahead_demand_forecast.csv')
df_margin_forecast = pd.read_csv('D:/Users/paulh\Desktop/Domäneprojekt2/Energy_production_price_prediction/margin_forecast.csv')
df_margin_forecast.forecast_date = pd.to_datetime(df_margin_forecast.forecast_date)
df_day_ahead_demand.timestamp_utc = pd.to_datetime(df_day_ahead_demand.timestamp_utc)

In [239]:
df_api_new_merged3 = pd.merge(df_api_new_merged2,df_day_ahead_demand, left_on='timestamp_utc', right_on='timestamp_utc', how='left')
df_api_new_merged3["date"] = df_api_new_merged3["timestamp_utc"].dt.date
df_api_new_merged3["date"] = pd.to_datetime(df_api_new_merged3["date"])
df_api_new_merged3 = pd.merge(df_api_new_merged3,df_margin_forecast, left_on='date', right_on='forecast_date', how='left')

In [241]:
df_api_new_merged3.dropna(inplace=True)
df_api_new_merged3 = df_api_new_merged3.groupby("timestamp_utc").last().reset_index()
df_api_new_merged3

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period_x,cos_hour,cos_day,1,2,3,...,settlement_date_y,settlement_period,boundary,publish_time_utc_x,transmission_system_demand,national_demand,date,forecast_date,publish_time_utc_y,margin
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.000000,19.0,17.0,15.0,...,2024-10-07,47,N,2024-10-07T20:45:00Z,25216.0,24297.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.000000,19.0,17.0,15.0,...,2024-10-07,48,N,2024-10-07T20:45:00Z,23960.0,23100.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.000000,19.0,17.0,15.0,...,2024-10-08,1,N,2024-10-07T20:45:00Z,23214.0,22480.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.000000,19.0,17.0,15.0,...,2024-10-08,2,N,2024-10-07T20:45:00Z,22858.0,22146.0,2024-10-07,2024-10-07,2024-10-05T21:00:00Z,8918.0
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.623490,18.0,16.0,14.0,...,2024-10-08,3,N,2024-10-07T20:45:00Z,22720.0,21970.0,2024-10-08,2024-10-08,2024-10-06T21:00:00Z,10251.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2024-10-10 19:30:00+00:00,92.43,82.15,1957.65,42.0,0.258819,-0.900969,19.0,17.0,14.0,...,2024-10-10,42,N,2024-10-10T17:47:00Z,34038.0,32558.0,2024-10-10,2024-10-10,2024-10-08T21:00:00Z,8844.0
91,2024-10-10 20:00:00+00:00,93.04,79.14,1992.70,43.0,0.500000,-0.900969,19.0,16.0,14.0,...,2024-10-10,43,N,2024-10-10T17:47:00Z,32550.0,31070.0,2024-10-10,2024-10-10,2024-10-08T21:00:00Z,8844.0
92,2024-10-10 20:30:00+00:00,89.29,79.14,2284.50,44.0,0.500000,-0.900969,19.0,16.0,14.0,...,2024-10-10,44,N,2024-10-10T17:47:00Z,30725.0,29245.0,2024-10-10,2024-10-10,2024-10-08T21:00:00Z,8844.0
93,2024-10-10 21:00:00+00:00,78.26,91.45,1793.20,45.0,0.707107,-0.900969,19.0,17.0,14.0,...,2024-10-10,45,N,2024-10-10T20:47:00Z,29012.0,27532.0,2024-10-10,2024-10-10,2024-10-08T21:00:00Z,8844.0


In [242]:
df_api_new_merged2.columns

Index(['timestamp_utc', 'market_price', 'day_ahead_price', 'volume',
       'settlement_period_x', 'cos_hour', 'cos_day', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', 'imbalance_price', 'market_price_lag96h',
       'imbalance_price_lag96h', 'day_ahead_price_lag1week', 'volume_lag96h',
       'generation_solar', 'installed_capacity_mwp', 'capacity_mwp',
       'settlement_date', 'settlement_period_y', 'boa', 'generation_wind',
       'imvalance_price_predictions', 'day_ahead_price_predictions',
       'Total_MW', 'Revenue_40', 'Revenue_30', 'Revenue_20', 'Revenue_10',
       'Revenue_60', 'Revenue_50', 'Revenue_70', 'Revenue_80', 'Revenue_90',
       'optimized_trade', 'revenue_normal', 'day_ahead_price_predictions_MLP',
       'imbalance_price_predictions_MLP', 'optimized_trade_MLP'],
      dtype='object')

In [248]:
combined_imbalance_X = df_api_new_merged3[["imbalance_price_predictions_MLP", "national_demand", "transmission_system_demand", "margin"]]
with open("scaler_MLP_extension_imbalance.pkl", "rb") as file:
    scaler = pickle.load(file)
combined_imbalance_X_scaled = scaler.transform(combined_imbalance_X.values)
combined_imbalance_X_scaled = torch.tensor(combined_imbalance_X_scaled, dtype=torch.float32)

model_imbalance = SimpleModel(input_dim=4)
model_imbalance.load_state_dict(torch.load("MLP_extension_imbalance.pth"))
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(combined_imbalance_X_scaled)
predictions = predictions.numpy()
df_api_new_merged3["imbalance_price_predictions_MLP_extension"] = predictions


c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\paulh\AppData\Local\Temp\ipykernel_15448\2423680995.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where

In [249]:
combined_day_ahead_X = df_api_new_merged3[["day_ahead_price_predictions_MLP", "national_demand", "transmission_system_demand", "margin"]]
with open("scaler_MLP_extension_day_ahead.pkl", "rb") as file:
    scaler = pickle.load(file)
combined_day_ahead_X_scaled = scaler.transform(combined_day_ahead_X.values)
combined_day_ahead_X_scaled = torch.tensor(combined_day_ahead_X_scaled, dtype=torch.float32)

model_day_ahead = SimpleModel(input_dim=4)
model_day_ahead.load_state_dict(torch.load("MLP_extension_day_ahead.pth"))
model_day_ahead.eval()
with torch.no_grad():
    predictions = model_day_ahead(combined_day_ahead_X_scaled)
predictions = predictions.numpy()
df_api_new_merged3["day_ahead_price_predictions_MLP_extension"] = predictions

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\paulh\AppData\Local\Temp\ipykernel_15448\952351878.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

In [251]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions_MLP_extension']
    Target_MW = row['5']
    imbalance_price = row['imbalance_price_predictions_MLP_extension']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged3['optimized_trade_MLP'] = df_api_new_merged3.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged3['revenue_normal'] = df_api_new_merged3['day_ahead_price'] * df_api_new_merged3['optimized_trade_MLP'] + \
                         (df_api_new_merged3['Total_MW'] - df_api_new_merged3['optimized_trade_MLP']) * \
                         (df_api_new_merged3['imbalance_price'] - 0.07 * (df_api_new_merged3['Total_MW'] - df_api_new_merged3['optimized_trade_MLP']))


In [252]:
df_api_new_merged3.revenue_normal.mean()

19855.437046491854